### Amanda Michelle Darwis - 0706022310051
### Bernardo Frederick Kowe - 0706022310049

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from matplotlib import pyplot as plt
import scipy.cluster.hierarchy as sch
from sklearn.cluster import AgglomerativeClustering, KMeans
from scipy.cluster import hierarchy
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.metrics import silhouette_score

In [12]:
!pip install pycountry

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 52.1 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [23]:
df = pd.read_csv("/content/drive/MyDrive/Data Mining/ALP/DataCoSupplyChainDataset.csv", encoding='latin1', low_memory=False)
df.head()

,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Id,Category Name,Customer City,...,Order Zipcode,Product Card Id,Product Category Id,Product Description,Product Image,Product Name,Product Price,Product Status,shipping date (DateOrders),Shipping Mode
0,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,2/3/2018 22:56,Standard Class
1,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/18/2018 12:27,Standard Class
2,CASH,4,4,-247.779999,309.720001,Shipping on time,0,73,Sporting Goods,San Jose,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/17/2018 12:06,Standard Class
3,DEBIT,3,4,22.860001,304.809998,Advance shipping,0,73,Sporting Goods,Los Angeles,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/16/2018 11:45,Standard Class
4,PAYMENT,2,4,134.210007,298.250000,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/15/2018 11:24,Standard Class


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180519 entries, 0 to 180518
Data columns (total 53 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Type                           180519 non-null  object 
 1   Days for shipping (real)       180519 non-null  int64  
 2   Days for shipment (scheduled)  180519 non-null  int64  
 3   Benefit per order              180519 non-null  float64
 4   Sales per customer             180519 non-null  float64
 5   Delivery Status                180519 non-null  object 
 6   Late_delivery_risk             180519 non-null  int64  
 7   Category Id                    180519 non-null  int64  
 8   Category Name                  180519 non-null  object 
 9   Customer City                  180519 non-null  object 
 10  Customer Country               180519 non-null  object 
 11  Customer Email                 180519 non-null  object 
 12  Customer Fname                

In [17]:
df.isnull().sum()

,0
Type,0
Days for shipping (real),0
Days for shipment (scheduled),0
Benefit per order,0
Sales per customer,0
Delivery Status,0
Late_delivery_risk,0
Category Id,0
Category Name,0
Customer City,0


In [7]:
print(df.shape)

(180519, 53)


In [8]:
df.describe()

,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Late_delivery_risk,Category Id,Customer Id,Customer Zipcode,Department Id,Latitude,...,Order Item Quantity,Sales,Order Item Total,Order Profit Per Order,Order Zipcode,Product Card Id,Product Category Id,Product Description,Product Price,Product Status
count,180519.000000,180519.000000,180519.000000,180519.000000,180519.000000,180519.000000,180519.000000,180516.000000,180519.000000,180519.000000,...,180519.000000,180519.000000,180519.000000,180519.000000,24840.000000,180519.000000,180519.000000,0.0,180519.000000,180519.0
mean,3.497654,2.931847,21.974989,183.107609,0.548291,31.851451,6691.379495,35921.126914,5.443460,29.719955,...,2.127638,203.772096,183.107609,21.974989,55426.132327,692.509764,31.851451,NaN,141.232550,0.0
std,1.623722,1.374449,104.433526,120.043670,0.497664,15.640064,4162.918106,37542.461122,1.629246,9.813646,...,1.453451,132.273077,120.043670,104.433526,31919.279101,336.446807,15.640064,NaN,139.732492,0.0
min,0.000000,0.000000,-4274.979980,7.490000,0.000000,2.000000,1.000000,603.000000,2.000000,-33.937553,...,1.000000,9.990000,7.490000,-4274.979980,1040.000000,19.000000,2.000000,NaN,9.990000,0.0
25%,2.000000,2.000000,7.000000,104.379997,0.000000,18.000000,3258.500000,725.000000,4.000000,18.265432,...,1.000000,119.980003,104.379997,7.000000,23464.000000,403.000000,18.000000,NaN,50.000000,0.0
50%,3.000000,4.000000,31.520000,163.990005,1.000000,29.000000,6457.000000,19380.000000,5.000000,33.144863,...,1.000000,199.919998,163.990005,31.520000,59405.000000,627.000000,29.000000,NaN,59.990002,0.0
75%,5.000000,4.000000,64.800003,247.399994,1.000000,45.000000,9779.000000,78207.000000,7.000000,39.279617,...,3.000000,299.950012,247.399994,64.800003,90008.000000,1004.000000,45.000000,NaN,199.990005,0.0
max,6.000000,4.000000,911.799988,1939.989990,1.000000,76.000000,20757.000000,99205.000000,12.000000,48.781933,...,5.000000,1999.989990,1939.989990,911.799988,99301.000000,1363.000000,76.000000,NaN,1999.989990,0.0


In [9]:
# Nilai tiap column

for col in df.columns:
    print(f"=== {col} ===")
    print(df[col].value_counts(dropna=False))
    print("\n")

=== Type ===
Type
DEBIT       69295
TRANSFER    49883
PAYMENT     41725
CASH        19616
Name: count, dtype: int64


=== Days for shipping (real) ===
Days for shipping (real)
2    56618
3    28765
6    28723
4    28513
5    28163
0     5080
1     4657
Name: count, dtype: int64


=== Days for shipment (scheduled) ===
Days for shipment (scheduled)
4    107752
2     35216
1     27814
0      9737
Name: count, dtype: int64


=== Benefit per order ===
Benefit per order
 0.000000      1177
 143.990005     199
 72.000000      194
 46.799999      188
 24.000000      181
               ... 
-212.410004       1
-18.959999        1
 177.169998       1
 130.979996       1
-196.179993       1
Name: count, Length: 21998, dtype: int64


=== Sales per customer ===
Sales per customer
122.839996    1264
109.190002    1247
116.989998    1243
120.889999    1243
123.489998    1243
              ... 
250.770004       1
163.779999       1
224.889999       1
380.160004       1
197.899994       1
Name: count, 

In [10]:
print(df.columns.tolist())

['Type', 'Days for shipping (real)', 'Days for shipment (scheduled)', 'Benefit per order', 'Sales per customer', 'Delivery Status', 'Late_delivery_risk', 'Category Id', 'Category Name', 'Customer City', 'Customer Country', 'Customer Email', 'Customer Fname', 'Customer Id', 'Customer Lname', 'Customer Password', 'Customer Segment', 'Customer State', 'Customer Street', 'Customer Zipcode', 'Department Id', 'Department Name', 'Latitude', 'Longitude', 'Market', 'Order City', 'Order Country', 'Order Customer Id', 'order date (DateOrders)', 'Order Id', 'Order Item Cardprod Id', 'Order Item Discount', 'Order Item Discount Rate', 'Order Item Id', 'Order Item Product Price', 'Order Item Profit Ratio', 'Order Item Quantity', 'Sales', 'Order Item Total', 'Order Profit Per Order', 'Order Region', 'Order State', 'Order Status', 'Order Zipcode', 'Product Card Id', 'Product Category Id', 'Product Description', 'Product Image', 'Product Name', 'Product Price', 'Product Status', 'shipping date (DateOrde

In [13]:
import pandas as pd
import pycountry

# 2. Dropping Columns (Redundan, Missing 100%, & Data Sensitif)
# Menambahkan 'Order Customer Id' dan 'Product Category Id' ke daftar drop
cols_to_drop = [
    'Product Description', 'Order Zipcode', 'Customer Email',
    'Customer Password', 'Product Image', 'Product Status',
    'Customer Street', 'Order Customer Id', 'Product Category Id'
]
df_clean = df.drop(columns=cols_to_drop)

# 3. Handling Missing Values pada data Customer
df_clean['Customer Lname'] = df_clean['Customer Lname'].fillna('')
df_clean['Customer Zipcode'] = df_clean['Customer Zipcode'].fillna(0)

# 4. Manual Mapping Khusus Customer Country (Mencegah NaN)
cust_country_map = {
    'EE. UU.': 'United States',
    'Puerto Rico': 'Puerto Rico'
}
df_clean['Customer Country'] = df_clean['Customer Country'].replace(cust_country_map)

# 5. Standardisasi Order Country Menggunakan pycountry
def standardize_order_country(name):
    manual_fixes = {
        'Estados Unidos': 'United States',
        'México': 'Mexico',
        'Corea del Sur': 'South Korea',
        'Rusia': 'Russian Federation',
        'Sáhara Occidental': 'Western Sahara'
    }
    if name in manual_fixes: return manual_fixes[name]
    try:
        # Fuzzy search untuk mencari nama Inggris standar
        result = pycountry.countries.search_fuzzy(name)
        return result[0].name
    except:
        return name

print("Memulai standardisasi order_country...")
order_unique = pd.Series(df_clean['Order Country'].unique())
order_mapping = {c: standardize_order_country(c) for c in order_unique}
df_clean['Order Country'] = df_clean['Order Country'].map(order_mapping)

# 6. Final Touch: Standardisasi Nama Kolom (Snake Case)
# Penting agar penulisan rumus di Tableau nanti konsisten dan mudah
df_clean.columns = [col.lower().replace(' ', '_').replace('(', '').replace(')', '') for col in df_clean.columns]

print("--- Cleaning Selesai ---")
print(f"Total Baris: {df_clean.shape[0]}")
print(f"Total Kolom: {df_clean.shape[1]}")
print("Data siap diimpor ke Tableau!")

Memulai standardisasi order_country...
--- Cleaning Selesai ---
Total Baris: 180519
Total Kolom: 44
Data siap diimpor ke Tableau!


In [15]:
df_clean.head()

,type,days_for_shipping_real,days_for_shipment_scheduled,benefit_per_order,sales_per_customer,delivery_status,late_delivery_risk,category_id,category_name,customer_city,...,order_item_total,order_profit_per_order,order_region,order_state,order_status,product_card_id,product_name,product_price,shipping_date_dateorders,shipping_mode
0,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,73,Sporting Goods,Caguas,...,314.640015,91.250000,Southeast Asia,Java Occidental,COMPLETE,1360,Smart watch,327.75,2/3/2018 22:56,Standard Class
1,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,73,Sporting Goods,Caguas,...,311.359985,-249.089996,South Asia,Rajastán,PENDING,1360,Smart watch,327.75,1/18/2018 12:27,Standard Class
2,CASH,4,4,-247.779999,309.720001,Shipping on time,0,73,Sporting Goods,San Jose,...,309.720001,-247.779999,South Asia,Rajastán,CLOSED,1360,Smart watch,327.75,1/17/2018 12:06,Standard Class
3,DEBIT,3,4,22.860001,304.809998,Advance shipping,0,73,Sporting Goods,Los Angeles,...,304.809998,22.860001,Oceania,Queensland,COMPLETE,1360,Smart watch,327.75,1/16/2018 11:45,Standard Class
4,PAYMENT,2,4,134.210007,298.250000,Advance shipping,0,73,Sporting Goods,Caguas,...,298.250000,134.210007,Oceania,Queensland,PENDING_PAYMENT,1360,Smart watch,327.75,1/15/2018 11:24,Standard Class


In [16]:
df_clean.isnull().sum()

,0
type,0
days_for_shipping_real,0
days_for_shipment_scheduled,0
benefit_per_order,0
sales_per_customer,0
delivery_status,0
late_delivery_risk,0
category_id,0
category_name,0
customer_city,0


In [84]:
# Nilai tiap column

for col in df_clean.columns:
    print(f"=== {col} ===")
    print(df_clean[col].value_counts(dropna=False))
    print("\n")

=== type ===
type
DEBIT       69295
TRANSFER    49883
PAYMENT     41725
CASH        19616
Name: count, dtype: int64


=== days_for_shipping_real ===
days_for_shipping_real
2    56618
3    28765
6    28723
4    28513
5    28163
0     5080
1     4657
Name: count, dtype: int64


=== days_for_shipment_scheduled ===
days_for_shipment_scheduled
4    107752
2     35216
1     27814
0      9737
Name: count, dtype: int64


=== benefit_per_order ===
benefit_per_order
 0.000000      1177
 143.990005     199
 72.000000      194
 46.799999      188
 24.000000      181
               ... 
-212.410004       1
-18.959999        1
 177.169998       1
 130.979996       1
-196.179993       1
Name: count, Length: 21998, dtype: int64


=== sales_per_customer ===
sales_per_customer
122.839996    1264
109.190002    1247
116.989998    1243
120.889999    1243
123.489998    1243
              ... 
250.770004       1
163.779999       1
224.889999       1
380.160004       1
197.899994       1
Name: count, Length: 

In [87]:
!pip install openpyxl

In [97]:
output_path = "/content/drive/MyDrive/Data Mining/ALP/DataCo_Clean_Final_For_Tableau.xlsx"

try:
    print("Sedang memproses penyimpanan ke Excel... Harap tunggu (jangan stop cell)")
    df_clean.to_excel(output_path, index=False, engine='openpyxl')
    print(f"✅ Berhasil! Data disimpan di: {output_path}")
except Exception as e:
    print(f"❌ Gagal menyimpan ke Excel: {e}")
    print("Saran: Gunakan .to_csv jika Excel terlalu berat bagi RAM Colab.")

Sedang memproses penyimpanan ke Excel... Harap tunggu (jangan stop cell)


Exception ignored in: <function ZipFile.__del__ at 0x7b000d704c20>
Traceback (most recent call last):
  File "/usr/lib/python3.12/zipfile/__init__.py", line 1966, in __del__
    self.close()
  File "/usr/lib/python3.12/zipfile/__init__.py", line 1983, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file


✅ Berhasil! Data disimpan di: /content/drive/MyDrive/Data Mining/ALP/DataCo_Clean_Final_For_Tableau.xlsx
